In [1]:
import os
os.chdir('../')
%pwd

'h:\\Personal Projects\\Indian-Housing-Prices-Backend'

In [2]:
import pandas as pd

In [4]:
data = pd.read_csv("artifacts/data_ingestion/Combined_Cities.csv")
data.head()

,house_type,location,price,numBathrooms,numBalconies,isNegotiable,verificationDate,SecurityDeposit,Status,bhk,house_size_sqft,city
0,0,27,9.998798,1.0,2.0,0,1.0,0,2,1,400,Delhi
1,0,9,9.903488,1.0,2.0,0,9.0,0,2,1,400,Delhi
2,1,75,9.047821,1.0,2.0,0,12.0,0,1,2,500,Delhi
3,2,9,10.778956,3.0,2.0,0,365.0,0,2,3,1020,Delhi
4,0,4,9.903488,2.0,2.0,0,365.0,0,0,2,810,Delhi


In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataValidationConfig:
    root_dir: Path
    STATUS_FILE: str
    unzip_data_dir: Path
    all_schema: dict

In [6]:
from houseProject.constants import *
from houseProject.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_validation_config(self) -> DataValidationConfig:
        config = self.config.data_validation
        schema = self.schema.COLUMNS

        create_directories([config.root_dir])

        data_validation_config = DataValidationConfig(
            root_dir=config.root_dir,
            STATUS_FILE=config.STATUS_FILE,
            unzip_data_dir = config.unzip_data_dir,
            all_schema=schema,
        )

        return data_validation_config

In [8]:
import os
from houseProject import logger

In [9]:
class DataValiadtion:
    def __init__(self, config: DataValidationConfig):
        self.config = config


    def validate_all_columns(self)-> bool:
        try:
            validation_status = None

            data = pd.read_csv(self.config.unzip_data_dir)
            all_cols = list(data.columns)

            all_schema = self.config.all_schema.keys()

            
            for col in all_cols:
                if col not in all_schema:
                    validation_status = False
                    with open(self.config.STATUS_FILE, 'w') as f:
                        f.write(f"Validation status: {validation_status}")
                else:
                    validation_status = True
                    with open(self.config.STATUS_FILE, 'w') as f:
                        f.write(f"Validation status: {validation_status}")

            return validation_status
        
        except Exception as e:
            raise e

In [10]:
try:
    config = ConfigurationManager()
    data_validation_config = config.get_data_validation_config()
    data_validation = DataValiadtion(config=data_validation_config)
    data_validation.validate_all_columns()
except Exception as e:
    raise e

[2025-05-18 22:03:27,099: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-05-18 22:03:27,117: INFO: common: yaml file: params.yaml loaded successfully]
[2025-05-18 22:03:27,117: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-05-18 22:03:27,117: INFO: common: created directory at: artifacts]
[2025-05-18 22:03:27,117: INFO: common: created directory at: artifacts/data_validation]
